# Collection setup and data load

## Get keys and urls

In [58]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_KEY = os.getenv("WEAVIATE_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_URL = os.getenv("OPENAI_URL")

print(f"Weaviate Key:{WEAVIATE_KEY}")
print(f"OpenAI API Key: {OPENAI_API_KEY}")
print(f"OpenAI URL: {OPENAI_URL}")



Weaviate Key:root-user-key
OpenAI API Key: sk-dummy-key-for-local-testing
OpenAI URL: http://host.docker.internal:11434


## Connect to Weaviate

You need to pass in your Weaviate Cloud URL and KEY.

In [59]:
import weaviate
from weaviate.classes.init import Auth

# Connect to the local instance
client = weaviate.connect_to_local(
  host="127.0.0.1", # the address to the learner's instance
  port=8080,
  grpc_port=50051,
  auth_credentials=Auth.api_key(WEAVIATE_KEY),
  headers={
    "X-OpenAI-Api-Key": OPENAI_API_KEY
  }
)

print(client.is_ready())

True


## Create a collection with a vectorizer

* [Weaviate Docs - collection creation and configuration](https://weaviate.io/developers/weaviate/manage-data/collections)
* [Weaviate integrated embedding models](https://weaviate.io/developers/weaviate/model-providers/weaviate/embeddings)

Examples of other embedding models:
* [Cohere](https://weaviate.io/developers/weaviate/model-providers/cohere/embeddings)
* [HuggingFace 🤗](https://weaviate.io/developers/weaviate/model-providers/huggingface/embeddings)
* [Ollama (self-hosted)](https://weaviate.io/developers/weaviate/model-providers/ollama/embeddings)
* [OpenAI](https://weaviate.io/developers/weaviate/model-providers/openai/embeddings)

In [40]:
from weaviate.classes.config import Configure

if client.collections.exists("Jeopardy"):
    client.collections.delete("Jeopardy")

# Create a collection - with Weaviate vectorizer
client.collections.create(
    name="Jeopardy",

    # Using local ollama embedding model
    vector_config=Configure.Vectors.text2vec_ollama(
        model="nomic-embed-text",
        api_endpoint=OPENAI_URL
    ),
)

## Import data
### Sample Data

In [41]:
import json

with open("../jeopardy_tiny.json") as file:
    data_10 = json.load(file)

print(json.dumps(data_10[0:2], indent=2))

[
  {
    "Category": "SCIENCE",
    "Question": "This organ removes excess glucose from the blood & stores it as glycogen",
    "Answer": "Liver"
  },
  {
    "Category": "ANIMALS",
    "Question": "It's the only living mammal in the order Proboseidea",
    "Answer": "Elephant"
  }
]


### Insert Many

> `insert_many` is only used for inserting small batches of data - must complete within the timeout.

[Weaviate Docs - insert many](https://weaviate.io/developers/weaviate/manage-data/import)

In [42]:
# Insert data
jeopardy = client.collections.get("Jeopardy")
jeopardy.data.insert_many(data_10)

BatchObjectReturn(_all_responses=[UUID('260e4760-9f66-4c39-8e71-f1cbd66850ba'), UUID('ad785629-2861-4724-a164-5a5859e177c9'), UUID('9f9089f1-b8d1-4fa2-b975-6bba6b216bc9'), UUID('a1a58798-3f07-45cb-978d-3f261192c1d6'), UUID('ed2ff626-6306-4b47-846a-66b64cbca921'), UUID('aeba80e5-1ab8-434b-97e3-8ca6162e9f1c'), UUID('c7d1c7d7-8252-40a4-84da-b665133d9364'), UUID('5f438790-0189-46b9-97f1-e180e7f29fbb'), UUID('5d902af2-5356-48de-9531-e340db176756'), UUID('2379af85-0e12-4be3-bcd9-800ef0c9548a')], elapsed_seconds=0.5013501644134521, errors={}, uuids={0: UUID('260e4760-9f66-4c39-8e71-f1cbd66850ba'), 1: UUID('ad785629-2861-4724-a164-5a5859e177c9'), 2: UUID('9f9089f1-b8d1-4fa2-b975-6bba6b216bc9'), 3: UUID('a1a58798-3f07-45cb-978d-3f261192c1d6'), 4: UUID('ed2ff626-6306-4b47-846a-66b64cbca921'), 5: UUID('aeba80e5-1ab8-434b-97e3-8ca6162e9f1c'), 6: UUID('c7d1c7d7-8252-40a4-84da-b665133d9364'), 7: UUID('5f438790-0189-46b9-97f1-e180e7f29fbb'), 8: UUID('5d902af2-5356-48de-9531-e340db176756'), 9: UUID('2

### Data preview

In [43]:
# Show data preview
jeopardy = client.collections.get("Jeopardy")
response = jeopardy.query.fetch_objects(limit=4)

for item in response.objects:
    print(item.uuid, item.properties)

2379af85-0e12-4be3-bcd9-800ef0c9548a {'answer': 'Sound barrier', 'question': 'In 70-degree air, a plane traveling at about 1,130 feet per second breaks it', 'category': 'SCIENCE'}
260e4760-9f66-4c39-8e71-f1cbd66850ba {'answer': 'Liver', 'question': 'This organ removes excess glucose from the blood & stores it as glycogen', 'category': 'SCIENCE'}
5d902af2-5356-48de-9531-e340db176756 {'answer': 'the atmosphere', 'question': 'Changes in the tropospheric layer of this are what gives us weather', 'category': 'SCIENCE'}
5f438790-0189-46b9-97f1-e180e7f29fbb {'answer': 'DNA', 'question': 'In 1953 Watson & Crick built a model of the molecular structure of this, the gene-carrying substance', 'category': 'SCIENCE'}


In [44]:
# Show data preview - with vectors
jeopardy = client.collections.get("Jeopardy")
response = jeopardy.query.fetch_objects(
    limit=4,
    include_vector=True
)

for item in response.objects:
    print(item.properties)
    print(item.vector, '\n')

{'answer': 'Sound barrier', 'question': 'In 70-degree air, a plane traveling at about 1,130 feet per second breaks it', 'category': 'SCIENCE'}
{'default': [0.05415799096226692, 0.04904330149292946, -0.20311763882637024, -0.0005461001419462264, 0.03222251310944557, 0.06144142895936966, -0.025474553927779198, 0.036186568439006805, -0.022849932312965393, -0.04580424726009369, -0.012417429126799107, 0.006467184517532587, -0.018686041235923767, 0.06339855492115021, 0.057318128645420074, -0.025442305952310562, -0.001217968761920929, -0.10014927387237549, 0.013149109669029713, -0.03403489664196968, -0.019234446808695793, -0.07446111738681793, -0.05489122122526169, 0.008517375215888023, 0.04322664439678192, 0.027735142037272453, -0.036039747297763824, 0.014504053629934788, -0.04025600105524063, 0.05343007668852806, 0.03913275897502899, -0.05151074752211571, 0.005962639581412077, -0.05370404198765755, -0.05360748991370201, -0.045496705919504166, 0.0030917609110474586, 0.04011998698115349, 0.033

### Super quick query example

In [45]:
response = jeopardy.query.near_text(
    query="African animals",
    # query="weather",
    limit=2
)

for item in response.objects:
    print(item.properties)

{'answer': 'Antelope', 'question': 'Weighing around a ton, the eland is the largest species of this animal in Africa', 'category': 'ANIMALS'}
{'answer': 'Elephant', 'question': "It's the only living mammal in the order Proboseidea", 'category': 'ANIMALS'}


## A bit bigger example - 2k objects

### Load data

In [46]:
import json

with open("../wiki-2k.json") as file:
    data_2k = json.load(file)

print(json.dumps(data_2k[0:2], indent=2))

[
  {
    "text": "At this point in the siege, Lee's army had strengthened the Petersburg line. They dug breastworks out of rifle pits. At night, with pick and shovel, they then turned the breastworks into  deep trenches. Pointed stakes turned outwards were designed to break up any frontal attacks. The area between the two lines became a no man's land. The summer that year was hot and dry. Streams and springs were quickly drying up causing a water shortage on both sides. The siege was quickly becoming a stalemate.",
    "title": "Siege of Petersburg",
    "url": "https://simple.wikipedia.org/wiki/Siege%20of%20Petersburg",
    "wiki_id": "20231101.simple_550339_9"
  },
  {
    "text": "1944  Holocaust: Anne Frank and her family are placed on the last transport train from the Westerbork transit camp to Auschwitz.",
    "title": "September 3",
    "url": "https://simple.wikipedia.org/wiki/September%203",
    "wiki_id": "20231101.simple_8532_17"
  }
]


### Create a collection with Named Vectors and SourceProperties

In [47]:
from weaviate.classes.config import Configure, Property, DataType

def create_wiki_collection():
    if client.collections.exists("Wiki"):
        client.collections.delete("Wiki")

    # Create a collection here - with Weaviate vectorizer and define source properties
    client.collections.create(
        name="Wiki",

        vector_config=[
            Configure.Vectors.text2vec_ollama(
                name="main_vector",
                model="nomic-embed-text",
                api_endpoint=OPENAI_URL,
                source_properties=['title', 'text'] # which properties should be used to generate a vector
            )
        ],

        # Example: how to define property schema (Optional)
        # properties=[  
        #     Property(name="title", data_type=DataType.TEXT),
        #     Property(name="text", data_type=DataType.TEXT),
        #     Property(name="url", data_type=DataType.TEXT),
        #     Property(name="wiki_id", data_type=DataType.TEXT),
        # ],
    )

create_wiki_collection()

### Import data - 2k objects with Batch

Batch speeds up the import process by grouping objects to be added in bigger batch groups.

Batch creates an internal buffer to collect objects to be added.<br>
Each time the buffer count reaches `batch_size`, batch sends the new objects to Weaviate.

Types of batch:
* `dynamic` - let batch calculate the optimal batch_size based on detected latency
* `fixed_size` - provide a fixed batch_size
* `rate_limit` - limit the number of requests (per minute), useful for working with models with a rate limit

### Take 1 – import sample 100

In [48]:
from tqdm import tqdm

sample_100 = data_2k[0:100]

wiki = client.collections.get("Wiki")

with wiki.batch.dynamic() as batch:
    for item in tqdm(sample_100, total=len(sample_100), desc="Uploading"):
        batch.add_object(item)

print(f"Wiki count: {len(wiki)}")

Uploading: 100%|██████████| 100/100 [00:00<00:00, 8094.61it/s]


Wiki count: 100


In [49]:
# check for errors
if(len(wiki.batch.failed_objects)>0):
    print("Import complete with errors")
    for err in wiki.batch.failed_objects:
        print(err)
else:
    print("Import complete with no errors")

Import complete with no errors


### Take 2 – import sample 100 – with UUID

To avoid inserting duplicates, you can generate a UUID based on the whole object or a unique property.

In [50]:
from weaviate.util import generate_uuid5

print(generate_uuid5("This UUID is always the same"))
print(generate_uuid5("This UUID is always the same"))
print(generate_uuid5("This UUID is always the same"))
print("====================================")

print(generate_uuid5("This UUID is different"))
print(generate_uuid5("This UUID is different"))
print("====================================")

obj1 = { "title": "this is an object", "count": 1 }
obj2 = { "title": "this is an object", "count": 2 }
print(generate_uuid5(obj1))
print(generate_uuid5(obj2))


8d3441c0-c1d1-5859-8a5e-efce9e7d3bd8
8d3441c0-c1d1-5859-8a5e-efce9e7d3bd8
8d3441c0-c1d1-5859-8a5e-efce9e7d3bd8
09f975a6-0e62-565a-982e-e6ce148eac86
09f975a6-0e62-565a-982e-e6ce148eac86
c3c3ad32-fa65-5944-a021-415f8fda02af
4d0b77d3-4862-59bc-bf9f-9fe2b9bf89f0


In [51]:
# recreate the collection to start again
create_wiki_collection()

> Rerun the import script multiple times.

> Starting from the second run, the script should finish a lot faster, and the wiki count shouldn't increase.

In [52]:
from tqdm import tqdm
from weaviate.util import generate_uuid5

sample_100 = data_2k[0:100]

wiki = client.collections.get("Wiki")

with wiki.batch.fixed_size(batch_size=20, concurrent_requests=2) as batch:
    for item in tqdm(sample_100, total=len(sample_100), desc="Uploading"):
        id = generate_uuid5(item["wiki_id"])

        batch.add_object(
            item,
            uuid=id
        )

print(f"Wiki count: {len(wiki)}")

Uploading: 100%|██████████| 100/100 [00:00<00:00, 133.57it/s]


Wiki count: 100


### Take 2 - import the rest of the data - but break if multiple errors

In [53]:
from tqdm import tqdm
from weaviate.util import generate_uuid5

wiki = client.collections.get("Wiki")

with wiki.batch.fixed_size(batch_size=200, concurrent_requests=2) as batch:
    for item in tqdm(data_2k, total=len(data_2k), desc="uploading"):
        id = generate_uuid5(item["wiki_id"])
        batch.add_object(item, uuid=id)

        # Check number of errors while running
        if(batch.number_errors > 10):
            print("Errors during batch import")
            break

uploading: 100%|██████████| 2000/2000 [00:19<00:00, 103.80it/s]


### Check for errors

In [54]:
if(len(wiki.batch.failed_objects)>0):
    print("Import complete with errors")
    for err in wiki.batch.failed_objects:
        print(err)
else:
    print("Import complete with no errors")

Import complete with no errors


## Bonus - iterate through all collection data

The client has a built-in function that allows you to iterate through all collection data.

In [55]:
wiki = client.collections.get("Wiki")

counter = 100

for item in wiki.iterator():
    print(item.properties)

    if (counter == 0): break
    
    counter -= 1

{'text': 'Mission Hills is an urban residential community of the San Fernando Valley, within the city of Los Angeles, California.', 'title': 'Mission Hills, Los Angeles', 'wiki_id': '20231101.simple_512123_0', 'url': 'https://simple.wikipedia.org/wiki/Mission%20Hills%2C%20Los%20Angeles'}
{'text': 'Pavlov researched classical conditioning through the use of dogs and their natural ability to salivate, produce water in their mouths. Thorndike and Watson rejected looking at one\'s own conscious thoughts and feelings ("Introspection"). They wanted to restrict psychology to experimental methods. Skinner\'s research leant mainly on behavior shaping using positive reinforcement (rewards rather than punishments).', 'title': 'Behaviorism', 'wiki_id': '20231101.simple_260848_4', 'url': 'https://simple.wikipedia.org/wiki/Behaviorism'}
{'text': 'On 27 April 1970 he was selected as a cosmonaut and flew as Commander on Soyuz T-5. He retired on 31 October 1992 due to medical reasons after injuries he 

You can also get `vector embeddings`, by using `include_vector`.

In [56]:
counter = 10

for item in wiki.iterator(include_vector=True):
    print(item.properties)
    print(item.vector)

    if (counter == 0): break
    
    counter -= 1

{'text': 'Mission Hills is an urban residential community of the San Fernando Valley, within the city of Los Angeles, California.', 'title': 'Mission Hills, Los Angeles', 'wiki_id': '20231101.simple_512123_0', 'url': 'https://simple.wikipedia.org/wiki/Mission%20Hills%2C%20Los%20Angeles'}
{'main_vector': [0.058951012790203094, 0.0005350571591407061, -0.16708554327487946, 0.04747238755226135, -0.020396165549755096, 0.02771313302218914, 0.0019493828294798732, -0.002385116880759597, 0.02813483029603958, -0.018960265442728996, 0.010526870377361774, -0.04105892404913902, 0.06883788108825684, 0.017218047752976418, -0.06508810073137283, -0.04502822831273079, -0.028502486646175385, -0.007299448363482952, -0.00579375633969903, 0.027452880516648293, -0.013873046264052391, -0.0034230703022331, -0.01050561387091875, -0.06360425800085068, 0.10510376840829849, 0.06889962404966354, 0.05326471105217934, 0.013208776712417603, -0.03427588567137718, 0.026895413175225258, -0.011984300799667835, -0.04592579

## Close the client

In [57]:
client.close()